In [ ]:
print("Hello, World!")

In [ ]:
%pwd

In [ ]:
import os
os.chdir("../")
%pwd

In [ ]:
%pip install -qU pypdf langchain-pinecone pinecone-notebooks langchain_community PyPDF2

In [ ]:

file_path = "./Data/The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND.pdf"  # Relative path to the file

from PyPDF2 import PdfReader

reader = PdfReader(file_path)

In [ ]:
# Print metadata and content of the first page
extracted_text = ""
for page in reader.pages[:]:  # Limit to first 5 pages
    text = page.extract_text()
    if text:  # Check if text is not None
        extracted_text += text + "\n\n"

In [ ]:
# Extracted text
print(len(extracted_text))

In [ ]:
%pip install -qU langchain-text-splitters

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # Small chunk size to illustrate splitting
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)

# Splitting text into chunks
text_chunks = text_splitter.split_text(extracted_text)



In [ ]:
# created chunks
print(len(text_chunks))

In [ ]:
%pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
%pip install -qU langchain-pinecone pinecone-notebooks

In [ ]:
from dotenv import load_dotenv
load_dotenv()
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

In [ ]:
# Import the Pinecone library
from pinecone import Pinecone

# Initialize a Pinecone client with your API key
pc = Pinecone(api_key=PINECONE_API_KEY)

In [ ]:
import time

index_name = "medbot"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [ ]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=model)

In [ ]:
# Only use it to delete vectors
from pinecone import Pinecone

# Initialize Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY)

# Define your index name
index_name = "medbot"

# Connect to the index
index = pc.Index(index_name)

# Delete all vectors
index.delete(delete_all=True)

print("All vectors have been deleted from the index.")

In [ ]:
from langchain_core.documents import Document
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import SentenceTransformerEmbeddings

# Convert text chunks into Document objects
documents = [Document(page_content=text) for text in text_chunks]

# Initialize LangChain's SentenceTransformer wrapper
embedding_model = SentenceTransformerEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create Pinecone Vector Store and automatically add documents
vector_store = PineconeVectorStore.from_documents(
    documents=documents,
    index_name="medbot",
    embedding=embedding_model  # Pass the LangChain-compatible embedding model
)

print("Documents successfully added to the Pinecone index.")


In [43]:
from langchain_pinecone import PineconeVectorStore
from langchain_community.embeddings import HuggingFaceEmbeddings

# Initialize a compatible embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")  # Fix: Use HuggingFaceEmbeddings

# Load an existing Pinecone index
doc_search = PineconeVectorStore.from_existing_index(
    index_name="medbot",  # Ensure this index exists in your Pinecone account
    embedding=embedding_model  # Fix: Use the correct embedding model
)

In [44]:
# Create a retriever
retriever = doc_search.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [34]:
user_query = "What is the treatment for diabetes?"

In [45]:
# Retrieve documents related to "Lymphatic system"
retrieved_docs = retriever.invoke(user_query)


In [ ]:
from dotenv import load_dotenv
import os

load_dotenv(".env")  # Force load the .env file

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
print(f"GOOGLE_API_KEY: {GOOGLE_API_KEY}")  # Should print the API key


In [ ]:
%pip install -qU langchain-google-genai

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0.4,
    max_tokens=None,
    timeout=None,
    max_retries=3,
    google_api_key=GOOGLE_API_KEY
)


In [37]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a medical companion to help people with their health-related queries. "
            "You answer based on retrieved context. If you don't find the answer from retrieved data, "
            "reply with 'I am sorry, I don't have the answer to that question.' "
            "Keep your response concise and under 100 words.",
        ),
        ("human", "{input}"),
    ]
)


chain = prompt | llm


In [39]:

chatbot_response = chain.invoke(retrieved_docs).content
print(chatbot_response)

There is no cure for diabetes currently. However, it can be managed with treatments including medications like metformin, acarbose, and troglitizone.  Herbal remedies like fenugreek and bilberry may also help.  Treatment focuses on maintaining normal blood glucose levels and preventing long-term complications through monitoring diet, exercise, and blood glucose.
